In [1]:
from clearml import Task
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.models import mobilenet_v2
from tqdm import tqdm
import math


batch_size = 64
lr = 0.001
momentum = 0.9
epochs = 10

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

configuration_dict = {'batch_size': batch_size, 'lr': lr, 'momentum': momentum, 'epochs': epochs}
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
iterations_in_epoch = math.ceil(len(trainset) / batch_size)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()


        # input channel, output filter, kernel size
        self.conv1 = nn.Conv2d(3, 64, 5, padding=2)
        self.conv2 = nn.Conv2d(64, 128, 5, padding=2)
        self.conv3 = nn.Conv2d(128, 256, 3)

        self.BatchNorm2d1 = nn.BatchNorm2d(64)
        self.BatchNorm2d2 = nn.BatchNorm2d(128)
        self.BatchNorm2d3 = nn.BatchNorm2d(256)

        self.BatchNorm2d4 = nn.BatchNorm2d(120)



        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(p=0)

        self.fc1 = nn.Linear(256 * 3 * 3, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.BatchNorm2d1(self.pool(F.leaky_relu(self.conv1(x))))
        x = self.BatchNorm2d2(self.pool(F.leaky_relu(self.conv2(x))))
        x = self.BatchNorm2d3(self.pool(F.leaky_relu(self.conv3(x))))
        x = x.view(-1, 256 * 3 * 3)
        x = self.dropout(F.leaky_relu(self.fc1(x)))
        x = self.dropout(F.leaky_relu(self.fc2(x)))
        x = self.fc3(x)
        return x

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#net = mobilenet_v2()
#net.classifier = nn.Linear(1280, 10)
net = Net()
net.to(device)
next(net.parameters()).is_cuda

True

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum)

In [6]:
task_name = f'noMLP_batch{batch_size}_lr{lr}_epochs{epochs}_simple_convnet'
task = Task.init(project_name="mlp_parametrization", task_name=task_name)

configuration_dict = task.connect(
    configuration_dict
)
logger = task.get_logger()

ClearML Task: created new task id=e6eaf1da80cf479e92688cf0489221af
2023-08-11 14:05:56,301 - clearml.Task - INFO - Storing jupyter notebook directly as code
2023-08-11 14:05:56,385 - clearml.Repository Detection - WARNING - Can't get branch information for git repo in c:\Users\grk\git\parametrization
2023-08-11 14:05:56,411 - clearml.Repository Detection - WARNING - Can't get commit information for git repo in c:\Users\grk\git\parametrization
2023-08-11 14:05:56,510 - clearml.Repository Detection - WARNING - Can't get diff information for git repo in c:\Users\grk\git\parametrization
ClearML results page: https://app.clear.ml/projects/f88e6dc889294a42833934ab069b799f/experiments/e6eaf1da80cf479e92688cf0489221af/output/log


In [8]:
for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(tqdm(trainloader)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.to(device))
        loss = criterion(outputs, labels.to(device))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 2000 mini-batches
            logger.report_scalar(
                title="Loss",
                series="running_loss",
                iteration=(i + 1) + iterations_in_epoch * (epoch + 1),
                value=running_loss / 50,
            )
            running_loss = 0.0

100%|██████████| 782/782 [00:20<00:00, 38.65it/s]


In [9]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images.to(device))
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.cpu() == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')
logger.report_scalar(
    title="Metrics",
    series="accuracy",
    iteration=len(trainset),
    value=100 * correct // total,
)

Accuracy of the network on the 10000 test images: 73 %


In [ ]:
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images.to(device))
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: plane is 66.8 %
Accuracy for class: car   is 60.3 %
Accuracy for class: bird  is 43.1 %
Accuracy for class: cat   is 36.5 %
Accuracy for class: deer  is 55.0 %
Accuracy for class: dog   is 49.3 %
Accuracy for class: frog  is 63.3 %
Accuracy for class: horse is 65.0 %
Accuracy for class: ship  is 65.9 %
Accuracy for class: truck is 67.0 %
